In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 931.03it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1439.20it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=2, nWay=4, batchSize=8, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
from training.models.FOMAML import FOMAML

fomaml = FOMAML(outerLR=5e-5, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
fomaml.training_step(next(iter(train_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

losses are tensor([0.5183, 0.5480, 0.4841, 0.4798], grad_fn=<NllLossBackward0>) and loss is 2.030160665512085
losses are tensor([0.5064, 0.5428, 0.4777, 0.4692], grad_fn=<NllLossBackward0>) and loss is 1.9961217641830444
losses are tensor([0.4956, 0.5376, 0.4712, 0.4590], grad_fn=<NllLossBackward0>) and loss is 1.963428020477295
losses are tensor([0.4827, 0.5345, 0.4664, 0.4472], grad_fn=<NllLossBackward0>) and loss is 1.9307979345321655
losses are tensor([0.4731, 0.5282, 0.4587, 0.4386], grad_fn=<NllLossBackward0>) and loss is 1.8985846042633057
losses are tensor([0.4408, 0.4907, 0.5207, 0.4262], grad_fn=<NllLossBackward0>) and loss is 1.8783413171768188
losses are tensor([0.4284, 0.4789, 0.5138, 0.4188], grad_fn=<NllLossBackward0>) and loss is 1.8399250507354736
losses are tensor([0.4173, 0.4684, 0.5057, 0.4105], grad_fn=<NllLossBackward0>) and loss is 1.8018122911453247
losses are tensor([0.4056, 0.4571, 0.4982, 0.4027], grad_fn=<NllLossBackward0>) and loss is 1.76362943649292
losse

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


losses are tensor([0.3292, 0.3214, 0.3991, 0.3255], grad_fn=<NllLossBackward0>) and loss is 1.3752589225769043
losses are tensor([0.3086, 0.3130, 0.3887, 0.3079], grad_fn=<NllLossBackward0>) and loss is 1.3181613683700562
losses are tensor([0.2894, 0.3059, 0.3799, 0.2911], grad_fn=<NllLossBackward0>) and loss is 1.266303539276123
losses are tensor([0.2741, 0.2962, 0.3684, 0.2778], grad_fn=<NllLossBackward0>) and loss is 1.2164573669433594
losses are tensor([0.2597, 0.2871, 0.3570, 0.2651], grad_fn=<NllLossBackward0>) and loss is 1.168939471244812
losses are tensor([0.2787, 0.2910, 0.1823, 0.3130], grad_fn=<NllLossBackward0>) and loss is 1.0650256872177124
losses are tensor([0.2700, 0.2573, 0.1616, 0.3053], grad_fn=<NllLossBackward0>) and loss is 0.9941574335098267
losses are tensor([0.2588, 0.2314, 0.1461, 0.2943], grad_fn=<NllLossBackward0>) and loss is 0.9305151700973511
losses are tensor([0.2465, 0.2113, 0.1335, 0.2818], grad_fn=<NllLossBackward0>) and loss is 0.8731834292411804
los

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

losses are tensor([0.2830, 0.3204, 0.2123, 0.1113, 0.4193, 0.4463],
       grad_fn=<NllLossBackward0>) and loss is 1.7925889492034912
losses are tensor([0.2552, 0.2894, 0.1756, 0.0969, 0.3957, 0.4147],
       grad_fn=<NllLossBackward0>) and loss is 1.6274380683898926
losses are tensor([0.2320, 0.2630, 0.1490, 0.0859, 0.3719, 0.3827],
       grad_fn=<NllLossBackward0>) and loss is 1.4844284057617188
losses are tensor([0.2126, 0.2408, 0.1288, 0.0769, 0.3484, 0.3517],
       grad_fn=<NllLossBackward0>) and loss is 1.3592939376831055
losses are tensor([0.1955, 0.2220, 0.1130, 0.0694, 0.3260, 0.3227],
       grad_fn=<NllLossBackward0>) and loss is 1.2486417293548584


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


losses are tensor([0.0460, 0.1848, 0.1009, 0.3090, 0.2101, 0.3219],
       grad_fn=<NllLossBackward0>) and loss is 1.1725306510925293
losses are tensor([0.0406, 0.1785, 0.0858, 0.2836, 0.1999, 0.2972],
       grad_fn=<NllLossBackward0>) and loss is 1.08543062210083
losses are tensor([0.0365, 0.1704, 0.0747, 0.2624, 0.1890, 0.2760],
       grad_fn=<NllLossBackward0>) and loss is 1.0090223550796509
losses are tensor([0.0332, 0.1624, 0.0660, 0.2447, 0.1784, 0.2570],
       grad_fn=<NllLossBackward0>) and loss is 0.941743016242981
losses are tensor([0.0304, 0.1548, 0.0589, 0.2290, 0.1686, 0.2401],
       grad_fn=<NllLossBackward0>) and loss is 0.8818447589874268


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


losses are tensor([0.2541, 0.0314, 0.3233, 0.1270, 0.2657, 0.2142],
       grad_fn=<NllLossBackward0>) and loss is 1.215673804283142
losses are tensor([0.2355, 0.0271, 0.2983, 0.1049, 0.2429, 0.1899],
       grad_fn=<NllLossBackward0>) and loss is 1.0986073017120361
losses are tensor([0.2166, 0.0239, 0.2770, 0.0891, 0.2239, 0.1693],
       grad_fn=<NllLossBackward0>) and loss is 0.9997536540031433
losses are tensor([0.1998, 0.0214, 0.2573, 0.0774, 0.2065, 0.1514],
       grad_fn=<NllLossBackward0>) and loss is 0.9139213562011719
losses are tensor([0.1846, 0.0193, 0.2396, 0.0680, 0.1922, 0.1366],
       grad_fn=<NllLossBackward0>) and loss is 0.8404543399810791


In [5]:
import torch

x = torch.Tensor([1,2,3,4,5,6,7,3,4,5,3])
labels = torch.Tensor([1,2,2,2,1,2,1,1,2,1,1])
print(x[torch.where(labels == 1)[0]])
print(x[torch.where(labels == 1)[0]].mean(dim=0))

tensor([1., 5., 7., 3., 5., 3.])
tensor(4.)


In [6]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

tunable_layers = {str(l) for l in range(12)}
for name, param in model.named_parameters():
    print(param.grad)
#     if not set.intersection(set(name.split('.')), tunable_layers):
#         param.requires_grad = False


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [7]:
print(model.parameters().data())

AttributeError: 'generator' object has no attribute 'data'